In [1]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump

sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [11,25]}         # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Purchase costs at suppliers'

d_params = {'dist': 'log-normal', 'r_f_params': [3,1]}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,61]}         # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Environment check

In [2]:
verbose = True; agg = False

In [3]:
w = [1,0,0,0,0]; objectives = ["costs","climate","water","land","fossil"]

rewards = dict()
env_config = {'M':5,'K':7,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60}

stoch_rd_seed = 10; det_rd_seed = 10 # Random seeds
obj_weights = dict(zip(objectives,w))

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
inst_gen.generate_basic_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

# Resetting the environment
state = env.reset(inst_gen, return_state=True)

done = False; day = 1

while not done:

    ''' Objectives Pay-off Matrix and Normalization '''
    pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)

    ''' Purchase '''
    [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state, env, inst_gen, objs=obj_weights)
    
    ''' Routing '''
    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

    ''' Complete Action '''        
    action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

    if verbose:
        if day == 1: verbose_module.headers.print_simple_header(w)
        print(f"\n------- Day {day}")
        verbose_module.objectives_performance.show_normalization(inst_gen,env)
        verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_dec)
    
    state, rewards[day], done, real_action, _,  = env.step(action,inst_gen,aggregated = agg)
    
    day += 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-14

---------------------------------------------------------------------------------------------------------------------------------------
                                                           Optimizing [1, 0, 0, 0, 0]                                                            
---------------------------------------------------------------------------------------------------------------------------------------

------- Day 1
	        	     costs     		    climate    		     water     		     land      		    fossil     	
	costs*  	3.28e+04 (0.00) 	2.87e+07 (1.00) 	1.81e+05 (1.00) 	2.00e+06 (1.00) 	7.59e+06 (1.00) 
	climate*	5.61e+04 (1.00) 	7.72e+06 (0.00) 	7.46e+04 (0.01) 	8.21e+05 (0.00) 	3.00e+06 (0.03) 
	water*  	5.59e+04 (1.00) 	7.91e+06 (0.01) 	7.40e+04 (0.00) 	8.34e+05 (0.02) 	3.25e+06 (0.09) 
	land*   	5.60e+04 (1.00) 	7.77e+06 (0.00) 	7.44e+04 (0.00) 	8.16e+05 (0.00) 	3.03e+06 (0.04) 


In [5]:
rewards

{1: {'transport cost': {1: 207.95245813875164,
   2: 232.02671581277775,
   3: 171.02376474323344,
   4: 202.89722098761234,
   5: 83.7946380492865,
   6: 271.0723369107449,
   7: 306.23286535759337},
  'purchase cost': {1: 1536.0,
   2: 1713.0,
   3: 634.0,
   4: 1630.5479852879066,
   5: 240.0,
   6: 1692.0,
   7: 1774.0},
  'holding cost': {1: 0.0,
   2: 0.0,
   3: 642.72,
   4: 1269.52,
   5: 0.0,
   6: 2084.3199999999997,
   7: 864.49},
  'backorders cost': {1: 2130.006948002307,
   2: 160.25952606053156,
   3: 0.0,
   4: 0.0,
   5: 1179.03306434617,
   6: 0.0,
   7: 0.0},
  'climate': {1: 133413.80184,
   2: 253792.73440000002,
   3: 76609.33581840001,
   4: 122527.32209599086,
   5: 1113060.9235200002,
   6: 187292.87566720002,
   7: 225607.86237290004},
  'water': {1: 1450.50696,
   2: 2028.1918499999997,
   3: 839.4037848,
   4: 1829.0523501080127,
   5: 1876.5801600000002,
   6: 2684.360262,
   7: 1343.5056759000001},
  'land': {1: 16909.333199999997,
   2: 28646.820600000003

# Feasible Random Seeds Search

In [ ]:
verbose = True; agg = False; save = False

In [2]:
w = [1,0,0,0,0]; objectives = ["costs","climate","water","land","fossil"]

st_seeds = [randint(0,10000) for i in range(100)]; dt_seeds = [randint(0,10000) for i in range(100)]

seeds = list(); it = 0
while len(seeds) < 1 and it < 100:

    rewards = dict()
    env_config = {'M':5,'K':7,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60}

    stoch_rd_seed = st_seeds[it]; det_rd_seed = dt_seeds[it] # Random seeds
    obj_weights = dict(zip(objectives,w))

    # Creating instance generator object
    inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
    inst_gen.generate_basic_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

    # Resetting the environment
    state = env.reset(inst_gen, return_state=True)

    done = False; day = 1
    try:
        
        while not done:

            ''' Objectives Pay-off Matrix and Normalization '''
            pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)

            ''' Purchase '''
            [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state, env, inst_gen, objs=obj_weights)

            
            ''' Routing '''
            GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

            ''' Complete Action '''        
            action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

            if verbose:
                if day == 1: verbose_module.headers.print_simple_header(w)
                print(f"\n------- Day {day}")
                verbose_module.objectives_performance.show_normalization(inst_gen,env)
                verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_dec)
            
            state, rewards[day], done, real_action, _,  = env.step(action,inst_gen,aggregated = agg)
            
            day += 1
        

        seeds.append((stoch_rd_seed,det_rd_seed))
        print(f"{len(seeds)} New feasible seeds found, st = {stoch_rd_seed}, det = {det_rd_seed}")
        if save: verbose_module.export_results.export_rewards(w,len(seeds),rewards)
    
    except:
        
        print(f"Failed random seeds of iteration {it}, st = {stoch_rd_seed}, det = {det_rd_seed}")
        
    it += 1

if save:
    file = open("C:/Users/ari_r/OneDrive - Universidad de los Andes/1. MIIND/Tesis/Experimentos/seeds","wb")
    dump(seeds, file); file.close()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-14

---------------------------------------------------------------------------------------------------------------------------------------
                                                           Optimizing [1, 0, 0, 0, 0]                                                            
---------------------------------------------------------------------------------------------------------------------------------------

------- Day 1
	        	     costs     		    climate    		     water     		     land      		    fossil     	
	costs*  	1.84e+05 (0.00) 	5.41e+07 (1.00) 	2.72e+05 (1.00) 	3.10e+06 (1.00) 	1.20e+07 (1.00) 
	climate*	6.35e+05 (0.98) 	1.58e+07 (0.00) 	1.57e+05 (0.07) 	1.77e+06 (0.03) 	6.14e+06 (0.04) 
	water*  	6.42e+05 (1.00) 	2.73e+07 (0.30) 	1.48e+05 (0.00) 	1.77e+06 (0.03) 	7.24e+06 (0.22) 
	land*   	6.43e+05 (1.00) 	2.13e+07 (0.14) 	1.52e+05 (0.03) 	1.73e+06 (0.00) 	6.29e+06 (0.07) 


# Experiments

In [2]:
weights = [[0.5,0.125,0.125,0.125,0.125]]; objectives = ["costs","climate","water","land","fossil"]

file = open("C:/Users/ari_r/OneDrive - Universidad de los Andes/1. MIIND/Tesis/Experimentos/seeds","rb")
seeds = load(file); file.close()

verbose = False; agg = False
for w in weights:

    for it in range(len(seeds)):

        rewards = dict()
        env_config = {'M':5,'K':7,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60,'back_o_cost':2000}

        stoch_rd_seed, det_rd_seed = seeds[it]
        obj_weights = dict(zip(objectives,w))

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_basic_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        state = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        while not done:

            ''' Objectives Pay-off Matrix and Normalization '''
            pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)

            ''' Purchase '''
            [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state, env, inst_gen, objs=obj_weights)

            
            ''' Routing '''
            GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

            ''' Complete Action '''        
            action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

            if verbose:
                if day == 1: verbose_module.headers.print_simple_header(w)
                print(f"\n------- Day {day}")
                verbose_module.objectives_performance.show_normalization(inst_gen,env)
                verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_dec)
            
            state, rewards[day], done, real_action, _,  = env.step(action,inst_gen,aggregated = agg)
            
            day += 1

        verbose_module.export_results.export_rewards(w,it+1,rewards)
        print(f"Done {w} - {it}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-14
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 0
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 1
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 2
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 3
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 4
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 5
Done [0.5, 0.125, 0.125, 0.125, 0.125] - 6


AttributeError: Unable to retrieve attribute 'x'

In [3]:
rewards

{1: {'transport cost': {1: 224.6037938859227,
   2: 275.07315476565793,
   3: 282.707418168908,
   4: 173.66497758255494,
   5: 119.55954559628009,
   6: 250.27233361226587,
   7: 413.11877638841065},
  'purchase cost': {1: 1394.458185275998,
   2: 1692.9109499412464,
   3: 1435.109522159079,
   4: 1511.8631211296788,
   5: 476.08576799947707,
   6: 801.7706713585588,
   7: 1654.1903163006455},
  'holding cost': {1: 0.0,
   2: 469.3,
   3: 0.0,
   4: 416.64,
   5: 0.0,
   6: 0.0,
   7: 0.0},
  'backorders cost': {1: 80561.67258896011,
   2: 0.0,
   3: 42825.90350209666,
   4: 0.0,
   5: 16047.57530437056,
   6: 62054.15023493169,
   7: 153027.3073929974},
  'climate': {1: 456245.93081329606,
   2: 406159.9679822795,
   3: 227569.55411853755,
   4: 241431.62059285856,
   5: 2333641.755810754,
   6: 222675.10001358984,
   7: 439477.56629653194},
  'water': {1: 4960.415556630572,
   2: 3595.2205100939527,
   3: 1192.4062953314967,
   4: 3191.32263193725,
   5: 3934.4349684407343,
   6: 29